In [76]:
import requests
from bs4 import BeautifulSoup
import sqlite3 as lite
import time
import pandas as pd

In [127]:
BaseStr = "https://markets.ft.com/data/funds/tearsheet/holdings?s="

con = lite.connect('FossilFreeFunds.db')

Fund_list = ['PRILX', 'PRBLX', 'BAWAX', 'BAFWX', 'BIAWX', \
             'ETAGX', 'ETCGX', 'ETILX', 'ETGLX', 'PFPWX', \
             'PXE', 'FTXN', 'FCG', 'PXJ', 'IEO', 'JHME', \
             'FSENX', 'XLE', 'VDE', 'VENAX', 'JSAAX']

In [113]:
with con:
        cur=con.cursor()
        cur.execute("DROP TABLE IF EXISTS Holdings") 
        cur.execute('''CREATE TABLE Holdings(Fund TEXT, Sector TEXT, \
        Net_Assets INT, Category_Avg INT)'''); 
        for i in range(0, len(Fund_list)):
            Funds = Fund_list[i]
            print('Processing Fund:' + Funds)
            page = requests.get(BaseStr + Funds)             
            soup = BeautifulSoup(page.content, 'html.parser') 
            tablediv = soup.find('div', class_ = 'mod-weightings__sectors__table')
            weightings_table = tablediv.find('table', class_ = 'mod-ui-table mod-ui-table--colored')
            time.sleep(1)
            for line in weightings_table.find_all('tbody'):
                rows = line.find_all('tr')
                for row in rows:
                    cols = weightings_table.find_all('td')
                    if cols:
                        cols = [ele.text.strip() for ele in cols]
                        Fund = Funds
                        Sector = row.find('td', class_ = 'mod-ui-table__cell--colored').text.strip()
                        Net_Assets = row.find_all('td')[1].text
                        Category_Avg = row.find_all('td')[2].text
                        try: 
                            row1 = (Fund, Sector, Net_Assets, Category_Avg)
                            cur.execute('INSERT OR IGNORE INTO Holdings VALUES(?,?,?,?)', row1)
                        except: 
                            print('DB error ')
con.commit()
print('Complete.')

Processing Fund:PRILX
Processing Fund:PRBLX
Processing Fund:BAWAX
Processing Fund:BAFWX
Processing Fund:BIAWX
Processing Fund:ETAGX
Processing Fund:ETCGX
Processing Fund:ETILX
Processing Fund:ETGLX
Processing Fund:PFPWX
Processing Fund:PXE
Processing Fund:FTXN
Processing Fund:FCG
Processing Fund:PXJ
Processing Fund:IEO
Processing Fund:JHME
Processing Fund:FSENX
Processing Fund:XLE
Processing Fund:VDE
Processing Fund:VENAX
Processing Fund:JSAAX
Complete.


In [138]:
Qry = '''SELECT * FROM Holdings'''
df = pd.read_sql_query(Qry, con)
df

,Fund,Sector,Net_Assets,Category_Avg
0,PRILX,Technology,26.02%,23.69%
1,PRILX,Financial Services,15.11%,13.53%
2,PRILX,Communication Services,12.49%,8.90%
3,PRILX,Healthcare,12.24%,13.33%
4,PRILX,Industrials,8.98%,9.45%
...,...,...,...,...
189,JSAAX,Real Estate,7.67%,4.14%
190,JSAAX,Communication Services,5.86%,6.72%
191,JSAAX,Other,5.03%,22.65%
192,JSAAX,Consumer Defensive,4.61%,5.08%


In [162]:
Qry = '''SELECT * FROM Holdings 
        WHERE Fund == "PRILX" '''

df1 = pd.read_sql_query(Qry, con)
df1.head(10)

,Fund,Sector,Net_Assets,Category_Avg
0,PRILX,Technology,26.02%,23.69%
1,PRILX,Financial Services,15.11%,13.53%
2,PRILX,Communication Services,12.49%,8.90%
3,PRILX,Healthcare,12.24%,13.33%
4,PRILX,Industrials,8.98%,9.45%
5,PRILX,Consumer Cyclical,7.35%,10.68%
6,PRILX,Consumer Defensive,7.00%,6.57%
7,PRILX,Real Estate,4.69%,2.36%
8,PRILX,Basic Materials,4.13%,2.66%
9,PRILX,Other,0.00%,8.83%


In [163]:
Qry = '''SELECT * FROM Holdings 
        WHERE Fund == "PXE" '''

df2 = pd.read_sql_query(Qry, con)
df2.head(10)

,Fund,Sector,Net_Assets,Category_Avg
0,PXE,Energy,99.90%,89.65%
1,PXE,Utilities,0.00%,6.37%
2,PXE,Technology,0.00%,1.16%
3,PXE,Basic Materials,0.00%,1.09%
4,PXE,Other,0.00%,0.88%
5,PXE,Industrials,0.00%,0.81%
6,PXE,Real Estate,0.00%,0.03%
7,PXE,Communication Services,0.00%,0.01%


In [164]:
Qry = '''SELECT * FROM Holdings 
        WHERE Fund == "JSAAX" '''

df3 = pd.read_sql_query(Qry, con)
df3.head(10)

,Fund,Sector,Net_Assets,Category_Avg
0,JSAAX,Technology,15.83%,16.84%
1,JSAAX,Financial Services,15.45%,12.65%
2,JSAAX,Consumer Cyclical,10.62%,9.74%
3,JSAAX,Healthcare,9.22%,9.85%
4,JSAAX,Industrials,8.81%,8.64%
5,JSAAX,Real Estate,7.67%,4.14%
6,JSAAX,Communication Services,5.86%,6.72%
7,JSAAX,Other,5.03%,22.65%
8,JSAAX,Consumer Defensive,4.61%,5.08%
9,JSAAX,Basic Materials,3.50%,3.69%


In [157]:
Qry = '''SELECT * FROM Holdings 
        WHERE Sector == "Energy"
        ORDER BY Net_Assets DESC;'''

df4 = pd.read_sql_query(Qry, con)
df4.head(10)

,Fund,Sector,Net_Assets,Category_Avg
0,VDE,Energy,99.91%,89.65%
1,VENAX,Energy,99.91%,89.65%
2,PXE,Energy,99.90%,89.65%
3,FCG,Energy,99.90%,89.65%
4,PXJ,Energy,99.81%,89.65%
5,FTXN,Energy,99.77%,89.65%
6,IEO,Energy,99.63%,89.65%
7,XLE,Energy,99.56%,89.65%
8,JHME,Energy,99.54%,89.65%
9,FSENX,Energy,98.14%,89.65%


In [165]:
con.close()